In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle

In [72]:
# Read files

summary_df = pd.read_parquet("out/qc/qc_summary_df.parquet")
with open("out/qc/issue_examples.pickle", "rb") as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    issue_trips = pickle.load(f)

In [73]:
print(summary_df.shape)
print(summary_df.sum(axis=0))

(842, 6)
pid                         5503155
avg_speed_trips                   0
na_times_trips                    0
same_time_trips                 251
min_max_time_issue_trips          0
very_long_trips                 849
dtype: int64


In [74]:
summary_df.sort_values('very_long_trips', ascending=False).head(10)

,pid,avg_speed_trips,na_times_trips,same_time_trips,min_max_time_issue_trips,very_long_trips
382,19380,0,0,0,0,99
478,9368,0,0,0,0,48
439,9247,0,0,2,0,36
829,18414,0,0,2,0,33
489,9372,0,0,1,0,32
380,19379,0,0,2,0,24
404,7120,0,0,5,0,22
758,14134,0,0,1,0,21
742,14119,0,0,2,0,20
479,1178,0,0,0,0,20


In [ ]:
grouping = summary_df.groupby('pid').sum().sum(axis =1).reset_index()
grouping.rename(columns={0: 'errors'}, inplace=True)
grouping = grouping[grouping.errors > 0]

# num of issues trips as perc of processed
new_df = []
for row in grouping.itertuples():
    pid = row.pid
    trips_df = gpd.read_parquet(f"out/trips/trips_{pid}_full.parquet")
    total_trips = trips_df['unique_trip_vehicle_day'].nunique()
    new_row = {'pid': pid, 'total_trips': total_trips, 'issues': row.errors, 'perc': (row.errors/total_trips)*100}
    new_df.append(new_row)

errors_review = pd.DataFrame(new_df)

In [70]:
errors_review.sort_values('perc', ascending=False).head(10)

,pid,total_trips,issues,perc
197,19380,9147,99,1.082322
169,9379,139,1,0.719424
141,7904,149,1,0.671141
3,704,170,1,0.588235
170,9718,200,1,0.500000
90,5693,666,3,0.450450
125,7109,690,3,0.434783
164,9371,860,3,0.348837
163,9370,696,2,0.287356
150,8144,356,1,0.280899


Look at some specific issue trips

In [32]:
pid = 7120
for dict in issue_trips:
    if dict['pid'] == pid:
        print(dict['very_long_trips'])
        break

['627120.02387813952222493780312023-04-29', '627120.023878484010820280092023-01-23', '627120.023878485910820880522023-02-25', '627120.023878490710826580142023-04-28', '627120.023878493610824580052023-06-01']


In [ ]:
trip = '627120.02387813952222493780312023-04-29'

df1 = gpd.read_parquet(f"out/trips/trips_{pid}_full.parquet")
df1[(df1['unique_trip_vehicle_day'] == trip)].tail(25)